# From Keras to TensorfloLite

In [1]:
import tensorflow as tf
import cv2
import os
import zipfile
import numpy as np
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import pathlib

In [2]:
# set the folder path where is located the model 
DIR = './bin'
name_model = 'rps.h5'

# input size model
input_size = 150

## 1.0 Import the dataset

In [3]:
# download the training dataset in /tmp if you don't have it in your path
!wget --no-check-certificate \
    https://storage.googleapis.com/laurencemoroney-blog.appspot.com/rps.zip \
    -O /tmp/rps.zip

--2020-01-29 11:07:19--  https://storage.googleapis.com/laurencemoroney-blog.appspot.com/rps.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 216.58.206.80
Connecting to storage.googleapis.com (storage.googleapis.com)|216.58.206.80|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 200682221 (191M) [application/zip]
Saving to: '/tmp/rps.zip'

/tmp/rps.zip        100%[===================>] 191.38M   108MB/s    in 1.8s    

2020-01-29 11:07:21 (108 MB/s) - '/tmp/rps.zip' saved [200682221/200682221]



In [4]:
# extract the training 
local_zip = '/tmp/rps.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp/')
zip_ref.close()

In [5]:
rock_dir = os.path.join('/tmp/rps/rock')
paper_dir = os.path.join('/tmp/rps/paper')
scissors_dir = os.path.join('/tmp/rps/scissors')
rock_files = os.listdir(rock_dir)
paper_files = os.listdir(paper_dir)
scissors_files = os.listdir(scissors_dir)

print('total training rock images:', len(os.listdir(rock_dir)))
print('total training paper images:', len(os.listdir(paper_dir)))
print('total training scissors images:', len(os.listdir(scissors_dir)))

total training rock images: 840
total training paper images: 840
total training scissors images: 840


In [6]:
# select n_images from the three classes
n_images = 600

next_rock = [os.path.join(rock_dir, fname) 
                for fname in rock_files[:n_images]]
next_paper = [os.path.join(paper_dir, fname) 
                for fname in paper_files[:n_images]]
next_scissors = [os.path.join(scissors_dir, fname) 
                for fname in scissors_files[:n_images]]

imgs = next_rock + next_paper + next_scissors

In [7]:
# import the selected images with matplotlib :)
dataset = np.empty((n_images*3, input_size, input_size, 3))

for i in range(len(imgs)):
    img = mpimg.imread(imgs[i]) 
    img = cv2.resize(img, (input_size, input_size))
    dataset[i,:,:,:] = img[:,:,:3]

# 2.0 Convert to TFLite

In [8]:
# create a dataset generator
def representative_dataset_gen():
    for i in range(0, dataset.shape[0]):
        imgs = dataset[i:i+1]
        yield [imgs.astype('float32')]

In [9]:
# import the converter loading the model
converter = tf.compat.v1.lite.TFLiteConverter.from_keras_model_file(os.path.join(DIR, name_model))

In [10]:
# set the converter for a full integer quantization of weights and activations
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_dataset_gen
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.uint8
converter.inference_output_type = tf.uint8

In [11]:
# convert to TensorFlowLite
tflite_quant_model = converter.convert()

In [12]:
# save the 
tflite_model_file = pathlib.Path('./bin/rps.tflite')
tflite_model_file.write_bytes(tflite_quant_model)

3490096